# **지도 학습(Supervised Learning)으로 보스턴 주택 가격 회귀 모델 만들기**

In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_boston

# ImportError: `load_boston` has been removed from scikit-learn since version 1.2.
# --> 데이터 제공X

ImportError: ignored

## 데이터 불러오기

In [ ]:
boston = load_boston()

In [ ]:
# 필요한 데이터 추출하기  dataset.속성, dataset[배열] 모두 가능
data = boston.data
label = boston.target
columns = boston.feature_names

In [ ]:
data = pd.DataFrame(data, columns=columns)
data.head()

In [ ]:
data.shape

## 데이터셋 준비하기

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(data, label, test_size=0.2, random_state=2023)
print(X_train['RM']) # 보스턴 집 방의 개수

## 모델링

### 1. 선형 회귀(Linear Regression)

### 1.1. 단순 선형 회귀(Simple Linear Regression)

In [ ]:
#1차원 배열 -> 2차원 배열로 바꿔주기 '(-1, 1)' 사용
# 차원 변경 후 5개만 확인해 보기
X_train['RM'].values.reshape(-1, 1)[:5]

In [ ]:
# train
sim_lr = LinearRegression()
sim_lr.fit(X_train['RM'].values.reshape((-1,1)) ,y_train)

## 모델 평가하기

In [ ]:
# 예측값. 방의 개수로 예측
y_pred = sim_lr.predict(X_test['RM'].values.reshape((-1,1)))
print(y_pred)

In [ ]:
print('단순 선형 회귀, R2: {:.4f}'.format(r2_score(y_test, y_pred)))
#.4f ===> 실수, 소수점 넷째자리까지

## 결과 시각화

In [ ]:
line_X = np.linspace(np.min(X_test['RM']), np.max(X_test['RM']), 10)
line_y = sim_lr.predict(line_X.reshape(-1,1))

# 산점도(scatter plot), 회귀선(regression line), 각주(legend) 나타내기
plt.scatter(X_test['RM'], y_test, s=10, c='black')
plt.plot(line_X, line_y, c='red')
plt.legend(['Regression line','Test data sample'], loc='upper left')

--> 너무 분산되어 나옴. 실제 값과 예측값의 차이多. Room 하나만 가지고 평가하기 부족.

## 모델링

### 1.2. 다중 선형 회귀(Multi Linear Regression)

In [ ]:
# Multiple Linear Regression
mul_lr = LinearRegression()

# 전체 columns를 사용
mul_lr.fit(X_train, y_train)

## 모델 평가하기

In [ ]:
y_pred = mul_lr.predict(X_test)

#y 테스트용 데이터, 예측치
print('다중 선형 회귀, R2: {:.4f}'.format(r2_score(y_test, y_pred)))

## 모델링

### 2. 결정 트리 모델(Decision Tree Regressor)

### 2.1. 단순 결정 트리 모델(Simple Decision Tree Regressor)

In [ ]:
dt_regr =DecisionTreeRegressor(max_depth=2)

In [ ]:
# train
# 1차원 배열 -> 2차원 배열로 바꿔주기
dt_regr.fit(X_train['RM'].values.reshape((-1,1)), y_train)

In [ ]:
#예측값
y_pred = dt_regr.predict(X_test['RM'].values.reshape(-1,1))

print('단순 결정 트리 회귀 R2: {:.4f}'.format(r2_score(y_test, y_pred)))

--> max_depth 변화를 줘 본다. 너무 높이면 학습 데이터와 오버피팅된다. 테스트용 데이터와 격차가 벌어짐.   
반복문으로 가장 좋은 depth를 찾는다.


In [ ]:
#배열 만들기
arr = np.arange(1,11)
print(arr)

best_depth = 1
best_r2 = 0
for depth in arr:
  dt_regr = DecisionTreeRegressor(max_depth=depth)
  dt_regr.fit(X_train['RM'].values.reshape((-1,1)), y_train)
  y_pred = dt_regr.predict(X_test['RM'].values.reshape(-1,1))

  temp_r2 = r2_score(y_test, y_pred)
  print('\n단순 결정 트리 회귀 depth={} R2: {:.4f}'.format(depth, temp_r2))

  if best_r2 < temp_r2:
    best_depth = depth
    best_r2 = temp_r2

print('최적의 결과는 depth={} r2={:.4f}'.format(best_depth, best_r2))

## 모델링

### 2.2. 다중 결정 트리 모델(Multi Decision Tree Regressor)

In [ ]:
dt_regr = DecisionTreeRegressor(max_depth=8)
dt_regr.fit(X_train, y_train)


#배열 만들기
arr = np.arange(1,11)
print(arr)

best_depth = 1
best_r2 = 0
for depth in arr:
  dt_regr = DecisionTreeRegressor(max_depth=depth)
  dt_regr.fit(X_train, y_train)
  y_pred = dt_regr.predict(X_test)

  temp_r2 = r2_score(y_test, y_pred)
  print('\n다중 결정 트리 회귀 depth={} R2: {:.4f}'.format(depth, temp_r2))

  if best_r2 < temp_r2:
    best_depth = depth
    best_r2 = temp_r2

print('최적의 결과는 depth={} r2={:.4f}'.format(best_depth, best_r2))

In [ ]:
# # 예측값
# y_pred = dt_regr.predict(X_test)
# print('다중 결정 트리 R2: {:.4f}'.format(r2_score(y_test, y_pred)))

### 3. 서포트 벡터 머신 회귀(Support Vector Machine Regressor)

### 3.1. 단순 서포트 벡터 머신 회귀

In [ ]:
# train
svm_regr = SVR(C=1)

svm_regr.fit(X_train['RM'].values.reshape(-1,1), y_train)

## 모델 평가하기

In [ ]:
# 예측값
y_pred = svm_regr.predict(X_test['RM'].values.reshape(-1,1))

print('단순 서포트 벡터 머신 회귀 R2: {:.4f}'.format(r2_score(y_test,y_pred)))

## 학습 결과 시각화하기

In [ ]:
# X축, y축 지정
line_X = np.linspace(np.min(X_test['RM']), np.max(X_test['RM']), 100)
line_y = svm_regr.predict(line_X.reshape(-1,1))

# 산점도(scatter plot), 회귀선(regression line), 각주(legend) 나타내기
plt.scatter(X_test['RM'], y_test, c='black')
plt.plot(line_X, line_y, c='red')
plt.legend(['Regression line', 'Test data sample'], loc='upper left')

### 3.2. 다중 서포트 벡터 머신 회귀

In [ ]:
svm_regr = SVR(C=20)
svm_regr.fit(X_train, y_train)
y_pred = svm_regr.predict(X_test)
print('다중 서포트 벡터 머신 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

In [ ]:
# 배열 만들기
arr = np.arange(10000,25000)
arr

In [ ]:
# 최적의 c값 찾기
best_C = 0
best_r2 = 0

for C in arr:
  svm_regr = SVR(C=C)
  svm_regr.fit(X_train, y_train)
  y_pred = svm_regr.predict(X_test)
  temp_r2 = r2_score(y_test, y_pred)
  #print('다중 서포트 벡터 머신 회귀, R2 : {:.4f}'.format(r2_score(y_test, y_pred)))

  if best_r2 < temp_r2:
    best_depth = depth
    best_r2 = temp_r2

print('최적의 결과는 depth={} r2={:.4f}'.format(best_depth, best_r2))

## 모델링

### 4. 멀티 레이어 퍼셉트론 회귀(Multi Layer Perceptron Regressor)

In [ ]:
# train
mlp_regr = MLPRegressor(solver='adam',hidden_layer_sizes=100)
# solver: lbfgs, sgd, adam 등

mlp_regr.fit(X_train, y_train)

In [ ]:
# 예측값
y_pred = mlp_regr.predict(X_test)

print('다중 MLP 회귀, R2: {:.4f}'.format(r2_score(y_test,y_pred)))

In [ ]:
# train
#max_iter 추가

mlp_regr = MLPRegressor(solver='adam',hidden_layer_sizes=100,max_iter=1000)
# solver: lbfgs, sgd, adam 등

mlp_regr.fit(X_train, y_train)

In [ ]:
# 예측값
y_pred = mlp_regr.predict(X_test)

print('다중 MLP 회귀, R2: {:.4f}'.format(r2_score(y_test,y_pred)))